# 第二章　パーセプトロン

## Introduction

機械学習は様々な分野に応用されていますが、いずれの場合も **入力** に対して **出力** があります。例えば、
1. 画像認識では画像が **入力** で画像に対する判定(犬なのか猫なのかなど)が**出力**
2. CTR推定ではBidRequestが **入力** になりCTR推定値が **出力**

機械学習で行っていることは、この入力から出力を計算する"規則"を大量のデータから決定することです。
機械学習には様々な種類がありますが、違いはこの”規則”として何を使っているかという点だけです。ディープラーニングではニューラルネットワークと呼ばれるネットワーク構造を使います。

[ゼロから作るDeep Learning] ( 以下、テキストと呼ぶ)の第二章では入力から出力を計算する簡単なモデルであるパーセプトロンを使ってANDゲートやORゲートを作る問題を考えます。上の**入力**と**出力**に対応させて考えると、
3. 二つのビットが**入力**で、一つのビットが**出力**
になります。

# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from ipywidgets import interact

## 2.1 パーセプトロンとは

テキスト図2-1参照

２つの入力$x_1, x_2$とパラメータ$w_1, w_2, \theta$から一つの出力$y$を計算する2入力パーセプトロンを定義します。

In [2]:
def two_perceptron(w1, w2, theta):
    def __func__(x1, x2):
        if(x1*w1+x2*w2<=theta):
            return 0
        else:
            return 1
    return __func__

def print_result(perceptron):
    # print all results calculated from perceptron
    print("x1, x2, y")
    for (x1, x2) in product([0, 1], [0, 1]):
        print("{},   {},   {}".format(x1, x2, perceptron(x1, x2)))        

## 2.2　単純な論理回路

### ANDゲート

パラメータを適切に設定するとANDゲートを再現できます。方法は一つではなく無限個あります。

In [3]:
w1 = 1
w2 = 1
theta = 1
AND = two_perceptron(w1, w2, theta)
print_result(AND)

x1, x2, y
0,   0,   0
0,   1,   0
1,   0,   0
1,   1,   1


### ORゲート

In [4]:
w1 = 1
w2 = 1
theta = 0
OR = two_perceptron(w1, w2, theta)
print_result(OR)

x1, x2, y
0,   0,   0
0,   1,   1
1,   0,   1
1,   1,   1


# 2.3.3 重みとバイアスによる実装

後の拡張に備えてしきい値$\theta$に代わりバイアス$b$を用います。ベクトルを使ってパーセプトロンを再実装します。

In [5]:
def two_perceptron_bias(w1, w2, b):
    def __func__(x1, x2):
        xs = np.array([x1, x2])
        ws = np.array([w1, w2])
        if(np.sum(xs*ws)+b<=0):
            return 0
        else:
            return 1
    return __func__

def print_result_bias(p):
    print("x1, x2, y")
    for (x1, x2) in product([0, 1], [0, 1]):
        print("{},   {},   {}".format(x1, x2, p(x1, x2)))        

ANDやORの計算は先と同様に可能です。

In [6]:
AND = two_perceptron_bias(1, 1, -1.5)
print("AND")
print_result_bias(AND)
OR = two_perceptron_bias(1, 1, -0.5)
print("\nOR")
print_result_bias(OR)
NAND = two_perceptron_bias(-1, -1, 1.5)
print("\nNAND")
print_result_bias(NAND)

AND
x1, x2, y
0,   0,   0
0,   1,   0
1,   0,   0
1,   1,   1

OR
x1, x2, y
0,   0,   0
0,   1,   1
1,   0,   1
1,   1,   1

NAND
x1, x2, y
0,   0,   1
0,   1,   1
1,   0,   1
1,   1,   0


ここで、パーセプトロンの動作を図示してみます。一般に$w_1x_1+w_2x_2+b=0$は($x_1, x_2$)平面の上で直線を表現します。なので、パーセプトロンの二値は直線で分断された領域をそれぞれ表現しています。

In [7]:
def widget_two_perceptron(result_gate):
    def f(w1, w2, b):
        xmin = -1
        xmax =  2
    
        x1s = np.linspace(xmin, xmax)
        x2s = (-w1*x1s-b)/w2
        plt.plot(x1s, x2s, "k", lw=2)
        plt.fill_between(x1s, x2s, np.linspace(xmin-1, xmin-1), color="pink")
        plt.plot([-1, 3], [0, 0], "k--")
        plt.plot([0, 0], [-1, 3], "k--")
    
        for (i, j) in product([0, 1], [0, 1]):
            if result_gate[i][j] == 0:
                c = "ro"
            else:
                c = "b^"
            plt.plot([i], [j], c)
    
        plt.xlim(xmin, xmax)
        plt.ylim(xmin, xmax)
        plt.xlabel("$x_1$")
        plt.ylabel("$x_2$")

    interact(f, w1=(0.25, 2, 0.25), w2=(0.25, +2, 0.25), b=(-2, 2, 0.5))

下の図は、テキストの図2.6に対応する図でORに対するパーセプトロンの可視化を行なっています。

In [8]:
print("OR")
widget_two_perceptron([[0, 1], [1, 1]])

OR


interactive(children=(FloatSlider(value=1.0, description='w1', max=2.0, min=0.25, step=0.25), FloatSlider(valu…

青は1を赤は0を表現している。

ORの点の分布の形から、適切にパラメータを設定すれば、単一の直線で0(赤)と1(青)を分割できる。これが、OR回路をパーセプトロンで表現できる要因です。

AND回路は？

次にXOR回路をパーセプトロンで表現することを考える。先と同様に、$(x_1, x_2)$平面で考える。

In [9]:
print("XOR")
widget_two_perceptron([[0, 1], [1, 0]])

XOR


interactive(children=(FloatSlider(value=1.0, description='w1', max=2.0, min=0.25, step=0.25), FloatSlider(valu…

どような直線を用いても、XORの0と1を分割することはできません。

## 2.5.2 XORゲートの実装

デジタル回路ではよく知られているように、ANDとORを組み合わせてXORを表現できます。今回のパーセプトロンの場合には次のように既存のパーセプトロンを組み合わせることで,XORが表現できます。

In [10]:
AND = two_perceptron_bias(1, 1, -1)
OR = two_perceptron_bias(1, 1, 0)
def XOR(x1, x2):
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y
print_result_bias(XOR)

x1, x2, y
0,   0,   0
0,   1,   1
1,   0,   1
1,   1,   0


これは、複数のゲートを組み合わせているので最初の一層のパーセプトロンに対して多層パーセプトロンと呼ばれます。テキストの図2-13を参照。

多層パーセプトロンが示す領域を体感するために、第一層のパーセプトロンをNANDおよびORに固定して、第二層のパーセプトロンを変化させながら、出力が0になる領域を図示します。

In [14]:
def plot_gate(gate):
    for (i, j) in product([0, 1], [0, 1]):
        if gate[i][j] == 0:
            c = "ro"
        else:
            c = "b^"
        plt.plot([i], [j], c)
    
def widget_2layer_2perceptron(result_gate):
    def f(w11, w12, b1, w21, w22, b2):
        xmin = -1
        xmax =  2
        
        #         AND = two_perceptron_bias(w1, w2, b)
        #         AND = two_perceptron_bias(1, 1, -1.5)        
        gate1 = two_perceptron_bias(w11, w12, b1)
        gate2 = two_perceptron_bias(w21, w22, b2)
    
        xs = np.linspace(xmin, xmax)
        x12s = list(product(xs, xs))
        x1s = np.array([x1 for (x1, x2) in x12s])
        x2s = np.array([x2 for (x1, x2) in x12s])
        
        s1s = [gate1(x1, x2) for (x1, x2) in x12s]
        s2s = [gate2(x1, x2) for (x1, x2) in x12s]
        ys = np.array([AND(s1, s2) for (s1, s2) in zip(s1s, s2s)])
        
        x1s_0 = np.array(x1s)[ys==0]
        x2s_0 = np.array(x2s)[ys==0]
        
        plt.plot(x1s_0, x2s_0, "x", color="pink")
        plt.plot([xmin, xmax], [0, 0], "k--")
        plt.plot([0, 0], [xmin, xmax], "k--")
    
        plot_gate(result_gate)
    
        plt.xlim(xmin, xmax)
        plt.ylim(xmin, xmax)
        plt.xlabel("$x_1$")
        plt.ylabel("$x_2$")
        plt.show()

    interact(f, 
             w11=(0.5, 1.5, 0.5), w12=(0.5, 1.5, 0.5), b1=(-1.5, +0.5, 0.5), 
             w21=(-1.5, -0.5, 0.5), w22=(-1.5, -0.5, 0.5), b2=(0.5, 2.5, 0.5))
    
widget_2layer_2perceptron([[0, 1], [1, 0]])

interactive(children=(FloatSlider(value=1.0, description='w11', max=1.5, min=0.5, step=0.5), FloatSlider(value…